In [1]:
import json
import os
import glob

In [30]:
fname = glob.glob('../Notebooks/*meta')[0]
print(fname)

../Notebooks/test.meta


In [26]:
with open(fname,'r') as metafile:
    meta_data = json.load(metafile)

In [28]:
meta_data.keys()

dict_keys(['binaryfilename', 'channel_labels', 'data_format', 'experiment_metadata_str', 'fs', 'no_channels', 'no_time_points', 'start_timestamp_unix', 'transmitter_id', 'volts_per_bit'])

In [40]:

params = [
    {'name': 'Metadata', 'type': 'group', 'children':
        [{'name': name, 'type': str(type(meta_data[name]))[8:-2], 'value': meta_data[name]} for name in meta_data.keys()]
     }
]

In [41]:
params[0]['children']

[{'name': 'binaryfilename', 'type': 'str', 'value': 'test.bin'},
 {'name': 'channel_labels',
  'type': 'list',
  'value': ['channel_0', 'channel_1', 'channel_2', 'channel_3']},
 {'name': 'data_format', 'type': 'str', 'value': '>i4'},
 {'name': 'experiment_metadata_str',
  'type': 'str',
  'value': 'kainate_acid_hippo'},
 {'name': 'fs', 'type': 'int', 'value': 512},
 {'name': 'no_channels', 'type': 'int', 'value': 4},
 {'name': 'no_time_points', 'type': 'int', 'value': 1843200},
 {'name': 'start_timestamp_unix', 'type': 'int', 'value': 1564066727},
 {'name': 'transmitter_id', 'type': 'str', 'value': '001'},
 {'name': 'volts_per_bit', 'type': 'float', 'value': 1.940255363782247e-10}]

In [39]:
str(int)[8:-2]

'int'

In [42]:
meta_data

{'binaryfilename': 'test.bin',
 'channel_labels': ['channel_0', 'channel_1', 'channel_2', 'channel_3'],
 'data_format': '>i4',
 'experiment_metadata_str': 'kainate_acid_hippo',
 'fs': 512,
 'no_channels': 4,
 'no_time_points': 1843200,
 'start_timestamp_unix': 1564066727,
 'transmitter_id': '001',
 'volts_per_bit': 1.940255363782247e-10}

In [2]:
import numpy as np


In [40]:
a ={'seizures':np.array([[0,1],[4,5]]),
   'spikes':np.array([[10,20]])}


In [7]:
a.keys()

dict_keys(['seizures', 'spikes'])

In [9]:
'spikes' in a.keys()

True

In [24]:
a['seizures'][0] = []

ValueError: cannot copy sequence with size 0 to array axis with dimension 2

In [17]:
a

{'spikes': [[10, 20]]}

In [16]:
del a['seizures']

In [20]:
[times[0] for times in a['seizures']]

[0, 4]

In [28]:
type(a['seizures'])

numpy.ndarray

In [55]:
a['seizures'].tolist()

[[0, 1], [4, 5]]

In [33]:
json.dumps(a)

'{"seizures": [[0, 1], [4, 5]], "spikes": [[10, 20]]}'

In [42]:
dictionary = a

In [41]:
for element in a:
    print(a[element])

[[0 1]
 [4 5]]
[[10 20]]


In [52]:
shapes = [len(dictionary[label].shape) and dictionary[label].shape[1]==2 for label in dictionary]

In [50]:
all(shapes)

True

In [ ]:
json.load()

In [61]:
from '/home/mfpleite/PycharmProjects/pyecog2/annotations' import annotations

SyntaxError: invalid syntax (<ipython-input-61-96df2879419c>, line 1)

In [1]:
import sys

In [1]:
import json

class Annotations:
    def __init__(self, dictionary=None, fname=None):
        if dictionary is not None and self.check_dict(dictionary):
            self.annotations_dict = dictionary
            self.labels = self.annotations_dict.keys()
        elif fname is not None:
            self.import_from_json(fname)
        else:
            self.annotations_dict = {}
            self.labels = self.annotations_dict.keys()

    @staticmethod
    def check_dict(dictionary):  # try to check if dictionary structure is valid
        if type(dictionary) is not dict:
            return False
        # Now check that all labels have lists of lists of length 2, with numbers inside : TO DO
        if not all([type(dictionary[label]) == list for label in dictionary]):
            print('Annotations: invalid type for annotations dictionary')
            return False
        if not all([all([len(i) == 2 for i in dictionary[label]]) for label in dictionary]):
            print('Annotations: invalid length(s) for [start, end] of annotations')
            return False
        if not all([all([all([type(j) in [int, float] for j in i]) for i in dictionary[label]]) for label in dictionary]):
            print('Annotations: invalid data type for start and end of annotations')
            return False
        return True

    def add_annotation(self, label, tstart, tend):
        if label in self.labels:
            self.annotations_dict[label].append([tstart, tend])
        else:
            self.add_label(label)
            self.annotations_dict[label] = [[tstart, tend]]

    def delete_annotation(self, label, index):
        try:
            del self.annotations_dict[label][index]
        except IndexError:
            print('Annotation index is out of range')

    def delete_label(self, label):
        del self.annotations_dict[label]
        self.labels = self.annotations_dict.keys()

    def add_label(self, label):
        if label not in self.labels:
            self.annotations_dict[label] = []
            self.labels = self.annotations_dict.keys()
        else:
            print('Annotations: Label already exists')

    def set_annotation_times(self, label, index, tstart, tend):
        self.annotations_dict[label][index] = [tstart, tend]

    def get_all_annotation_times(self,label):
        return [i for i in self.annotations_dict[label]]

    def get_all_start_times(self, label):
        return [i[0] for i in self.annotations_dict[label]]

    def get_all_end_times(self, label):
        return [i[1] for i in self.annotations_dict[label]]

    def import_from_json(self, fname):
        with open(fname, 'r') as f:
            self.annotations_dict = json.load(f)
        if not self.check_dict(self.annotations_dict):
            self.annotations_dict = {}
            print('Annotations: invalid dictionary in Json file')
        self.labels = self.annotations_dict.keys()

    def export_to_json(self, fname):
        with open(fname, 'w') as f:
            json.dump(self.annotations_dict, f)

    def export_to_csv(self, fname, label):
        with open(fname, 'w') as f:
            f.write(label + ',' + 'start,stop\n')
            for i,j in enumerate(self.annotations_dict[label]):
                f.write(str(i) + ',' + str(j[0]) + ',' + str(j[1]) + '\n')

    def __repr__(self):
        return repr(self.annotations_dict)

    def __str__(self):
        return str(self.annotations_dict)

In [2]:
a = Annotations({'seizure':[[1,2],[5,6],[5,7]]})

In [5]:
a

{'seizure': [[1, 2], [5, 6], [5, 7], [4, 5.6], [7, 9]]}

In [6]:
a.add_annotation('seizure', 4,5.6)
a.add_annotation('seizure', 7,9)

In [7]:
b = a.get_all_annotation_times('seizure')[0]

In [18]:
b[0]= 0

In [134]:
a.annotations_dict

{'seizure': [[1, 2], [5, 6], [5, 7], [4, 5.6], [7, 9]]}

In [135]:
a

{'seizure': [[1, 2], [5, 6], [5, 7], [4, 5.6], [7, 9]]}

In [136]:
a.delete_annotation('seizure',1)

In [137]:
a.get_all_end_times('seizure')

[2, 7, 5.6, 9]

In [138]:
a.export_to_json('annotations_test.ano')

In [139]:
b = Annotations(fname='annotations_test.ano')

In [140]:
b.annotations_dict

{'seizure': [[1, 2], [5, 7], [4, 5.6], [7, 9]]}

In [141]:
a

{'seizure': [[1, 2], [5, 7], [4, 5.6], [7, 9]]}

In [142]:
a.export_to_csv('annotations_test.csv','seizure')

In [8]:
sum([4,5])

9

In [20]:
3600*500*3*8

43200000

In [21]:
import colorsys

In [30]:
c =array(colorsys.hls_to_rgb(0, .5, .9))*255

NameError: name 'array' is not defined

TypeError: list expected at most 1 arguments, got 3